In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/prediksi/prediksi_inset.csv", sep=";")

In [3]:
data

,Tweet,label,Tweet_Parsed,Prediksi,Prediksi_Score
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal pakai baterai kapasitas turun,baterai negatif,baterai -2
1,Kmera iphone 11 mmeng beda,kamera positif,kamera beda,kamera negatif,kamera -2
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera depannya jelek liat liat pengguna foto ...,kamera negatif,kamera -2
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,meluncurkan peningkatan daya tahan baterai leb...,baterai positif,baterai 3
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...",spesifikasi positif,rugi beli maksud langsung tidak speknya lebih ...,harga negatif,harga -7
...,...,...,...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif,tidak keren charger bawaannya belum cepat charger,baterai negatif,baterai -5
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif,yaelah belum cepat charger charge charger bawa...,baterai positif,baterai 2
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif,butuh waku charge charger bawaan daya kecil,baterai negatif,baterai -5
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif,udah mahal chargernya belum cepat charger kecewa,"harga positif,baterai positif","harga 1,baterai 0"


In [4]:
class eval:
    def __init__(self):
        pass
    
    def aspect_count(self,data, aspect, label_col = 'label'):
        label_list = data[label_col].values
        pos, neg = 0, 0
        for i in label_list:
            split = i.split(",")
            for j in split:
                in_split = j.split(" ")
                if in_split[0] == aspect:
                    if in_split[1] == 'negatif':
                        neg = neg+1
                    else:
                        pos = pos+1
        return pos,neg
    
    def eval_count(self,data,true_label_col='label',pred_label_col='Prediksi',aspect_list=['general','spesifikasi','kamera','baterai','harga']):
        label_list = [str(i).split(",") for i in data[true_label_col].values]
        pred_list = [str(i).split(",") for i in data[pred_label_col].values]
        sent = ['positif','negatif']
        eval_list = []
        for i in aspect_list:
            for j in range(len(sent)):
                asp_count = self.aspect_count(data,i)
                record = [i,sent[j],asp_count[j]]
                pred_count = 0
                pred_true = 0
                pred_false = 0
                for k,l in zip(label_list,pred_list):
                    if " ".join([i,sent[j]]) in l:
                        pred_count = pred_count+1
                        if " ".join([i,sent[j]]) in k:
                            pred_true = pred_true+1
                        else:
                            pred_false = pred_false+1
                record.extend([pred_count,pred_true,pred_false])
                eval_list.append(record)
        return eval_list
    
    def get_pred_text(self,data,true_label_col='label',pred_label_col='Prediksi',text_col="Tweet",prep_text_col='Tweet_Parsed',predicted = False):
        label_list = data[true_label_col].values
        pred_list = data[pred_label_col].values
        text_list = data[text_col].values
        prep_text_list = data[prep_text_col].values
        pred_text = []
        for i in range(data.shape[0]):
            if predicted == False:
                if label_list[i] != pred_list[i]:
                    pred_text.append([text_list[i],prep_text_list[i],label_list[i],pred_list[i]])
            else:
                if label_list[i] == pred_list[i]:
                    pred_text.append([text_list[i],prep_text_list[i],label_list[i],pred_list[i]])
                
        return pred_text

In [5]:
evaluate = eval()

In [6]:
eval_result = evaluate.eval_count(data)
eval_result = pd.DataFrame(eval_result,columns=['aspect','sentiment','label count','predicted count','true predict','false predict'])
eval_result = eval_result.groupby(['aspect','sentiment']).sum()
eval_result['percent_true'] = round((eval_result['true predict']/eval_result['label count'])*100,2)
eval_result['percent_predict_true'] = round((eval_result['true predict']/eval_result['predicted count'])*100,2)
eval_result

label count  predicted count  true predict  \
aspect      sentiment                                               
baterai     negatif             68              112            51   
            positif             80               50            26   
general     negatif             69              175            50   
            positif             99              107            48   
harga       negatif             65               50            19   
            positif             61               91            40   
kamera      negatif             67               85            38   
            positif            100               89            52   
spesifikasi negatif             64               29            16   
            positif             74               20             7   

                       false predict  percent_true  percent_predict_true  
aspect      sentiment                                                     
baterai     negatif               61         75.00                 45.54  
            positif               24         32.50                 52.00  
general     negatif              125         72.46                 28.57  
            positif               59         48.48                 44.86  
harga       negatif               31         29.23                 38.00  
            positif               51         65.57                 43.96  
kamera      negatif               47         56.72                 44.71  
            positif               37         52.00                 58.43  
spesifikasi negatif               13         25.00                 55.17  
            positif               13          9.46                 35.00

In [7]:
false_tweet = evaluate.get_pred_text(data)
false_tweet = pd.DataFrame(false_tweet,columns=['Tweet','Tweet_Parsed','True_Label','Pred_Label'])
false_tweet

,Tweet,Tweet_Parsed,True_Label,Pred_Label
0,Kmera iphone 11 mmeng beda,kamera beda,kamera positif,kamera negatif
1,"xs max, senderrr. tapi rugi sih kalo kamu beli...",rugi beli maksud langsung tidak speknya lebih ...,spesifikasi positif,harga negatif
2,Saya juga dulu confuse nak pilih iphone 11 or ...,bingung ingin pilih review spesifikasi kurang ...,kamera negatif,"general positif,kamera positif"
3,Hasil kamera xiaomi note 10 bisa di adulah sam...,hasil kamera xiaomi note bisa adulah lebih,kamera negatif,kamera positif
4,Kalo sender lebih mentingin ketajaman layar me...,lebih lebih ketajaman layar lebih keseluruhan ...,"baterai positif, spesifikasi positif",general positif
...,...,...,...,...
435,buat iphone 11 ini apple udah mengembangkan te...,udah mengembangkan teknologi khusus bisa menek...,baterai positif,baterai negatif
436,ip11 ini punya teknologi yang bisa menekan pen...,ip teknologi bisa menekan penggunaan daya sist...,baterai positif,"baterai negatif,spesifikasi negatif"
437,"yaelah belum fast charging, lama nih kalo ngec...",yaelah belum cepat charger charge charger bawa...,baterai negatif,baterai positif
438,udah mahal begini masa chargernya blm fast cha...,udah mahal chargernya belum cepat charger kecewa,baterai negatif,"harga positif,baterai positif"


In [8]:
true_tweet = evaluate.get_pred_text(data,predicted=True)
true_tweet = pd.DataFrame(true_tweet,columns=['Tweet','Tweet_Parsed','True_Label','Pred_Label'])
true_tweet

,Tweet,Tweet_Parsed,True_Label,Pred_Label
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",admin normal pakai baterai kapasitas turun,baterai negatif,baterai negatif
1,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera depannya jelek liat liat pengguna foto ...,kamera negatif,kamera negatif
2,Apple meluncurkan iPhone 11 dengan peningkatan...,meluncurkan peningkatan daya tahan baterai leb...,baterai positif,baterai positif
3,Persahabatan memang lebih indah kalau diabadik...,persahabatan lebih indah diabadikan kamera,kamera positif,kamera positif
4,Iphone 11 pun menarik,menarik,general positif,general positif
...,...,...,...,...
303,kenapa sih iphone tuh baterainya ga pernah ber...,tuh baterainya tidak bertahan,baterai negatif,baterai negatif
304,"baterai nya boros, ga cocok buat orang yang pe...",baterai boros tidak cocok orang pekerjaannya h...,baterai negatif,baterai negatif
305,ngecharge nya butuh watu lama ini iphone 11,charge butuh watu,baterai negatif,baterai negatif
306,ga keren banget masa charger bawaannya belum f...,tidak keren charger bawaannya belum cepat charger,baterai negatif,baterai negatif


In [9]:
eval_result.to_csv("../data/final/evalresult_inset.csv", sep=";", index=False)
false_tweet.to_csv("../data/final/falsetweet_inset.csv", sep=";", index=False)
true_tweet.to_csv("../data/final/truetweet_inset.csv", sep=";", index=False)